# Identificação de Objetos

Nesta atividade, vamos explorar como identificar objetos encontrando seus contornos a partir de segmentação de cores.

Detecção de contornos é uma técnica importante em visão computacional. A ideia é que os contornos são as bordas dos objetos, e portanto, podem ser usados para detectar a localização de objetos, além de outras métricas como centro, área, perímetro, etc.

Fonte das imagens: [http://time.com/4299724/coca-cola-diet-coke-redesign/](http://time.com/4299724/coca-cola-diet-coke-redesign/)

# Componentes conexos e contornos

Neste exercicio vamos trabalhar na segunte imagem. Nosso objetivo é encontrar os contornos da **parte vermelha** das latinhas de coca-cola.

![image.png](img/coke-cans.jpg)


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("img/coke-cans.jpg")
coke_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

Matplotlib is building the font cache; this may take a moment.


## Filtrando o objeto

Revisando o círculo de cores HSV, temos:

![](img/hsv_circle.png)

Conforme ilustrado acima, a cor vermelha aparece em duas regiões distintas do círculo HSV: uma entre 0 e 30 graus e outra entre 330 e 360 graus.

Uma limitação do OpenCV é sua incapacidade de realizar seleções em intervalos não contínuos diretamente, como seria necessário para a cor vermelha (-30 a 30 graus). Isso representa um desafio na segmentação eficaz do vermelho.

No entanto, uma abordagem viável é realizar duas seleções de intervalos separadas para cada faixa do vermelho, aplicar a segmentação em ambas e, posteriormente, unir as duas máscaras resultantes. Esta técnica permite a segmentação eficiente da cor vermelha, contornando a restrição de seleção de intervalos do OpenCV.

Abaixo, temos o código para realizar a segmentação da cor vermelha, neste código, combinamos as duas máscaras resultantes utilizando o operador lógico OR do OpenCV, que retorna branco se pelo menos um dos pixels for branco. A função `cv2.bitwise_or` recebe os seguintes parâmetros:

* `cv2.bitwise_or(src, src2, **mask)`
    * `src1`: primeira imagem, ou máscara, de entrada ou matriz real.
    * `src2`: segunda imagem, ou máscara, de entrada ou matriz real.
    * `mask` (opcional): máscara de entrada de 8 bits. A operação será realizada apenas nos elementos especificados pela máscara.

Também temos o operator lógico AND, que retorna branco se ambos os pixels forem brancos, na função `cv2.bitwise_and`.

Depois da execução do código abaixo, podemos notar que a segmentação resultante está muito boa, mas ainda há alguns ruídos.

In [2]:
cor_menor1 = np.array([172, 50, 50])
cor_maior1 = np.array([180, 255, 255])
mask_coke1 = cv2.inRange(coke_hsv, cor_menor1, cor_maior1)

cor_menor2 = np.array([0, 50, 50])
cor_maior2 = np.array([8, 255, 255])
mask_coke2 = cv2.inRange(coke_hsv, cor_menor2, cor_maior2)

mask_coke = cv2.bitwise_or(mask_coke1, mask_coke2)

cv2.imshow("image", img)
cv2.imshow("mask 1", mask_coke1)
cv2.imshow("mask 2", mask_coke2)
cv2.imshow("mask", mask_coke)
cv2.waitKey()
cv2.destroyAllWindows()


## Componentes conexos

Após a segmentação da imagem por mascaramento, podemos observar que os pixels de interesse podem formar um ou mais grupos conectados entre si, ou seja, conjuntos de pixels que se comunicam através de algum caminho que passa apenas por pixels de interesse (brancos)

No OpenCV, é possível encontrar componente conexos em imagens tons de cinza através da função `cv2.findContours()`. Ela considera pixels de valor maior do que 0 como pixels de interesse. 

Usamos a seguinte função para encontrar os contornos:

```
contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
```

onde:
* `mask` é a imagem com a máscara binária de entrada.
* `cv2.RETR_CCOMP` indica que queremos organizar os contornos em componentes conexos e buracos dentro deles - veja mais detalhes em [Contours Hierarchy](https://docs.opencv.org/4.x/d9/d8b/tutorial_py_contours_hierarchy.html).
* `cv2.CHAIN_APPROX_NONE` indica que queremos armazenar todos os pontos do contorno.
* `contours` é uma lista de contornos. Cada contorno é uma lista de pontos (x, y) que formam o polígono que delimita o contorno.
* `hierarchy` é uma lista indicando a organização dos contornos em termos dos componentes e de seus buracos.


Os componentes conexos são representados através de seus contornos internos, ou seja, dos pixels de cada componente conexo que são vizinhos a pixels de fundo. Para desenhar os contornos em uma imagem, usamos a função `cv2.drawContours()`, que usamos da forma:

```
cv2.drawContours(img, contours, indice, cor)
```

* `img` é a imagem colorida ou tons de cinza onde serão desenhados os contornos.
* `contours` é a lista de contornos obtida com `cv2.findContours()`, ou seja, recebe uma lista de lista. Então assumindo que `contours[i]` seja um contorno, a função esperaria uma sintaxe como `cv2.drawContours(img, [contours[i]], indice, cor)`.
* `indice` é o índice do contorno dentro da lista a ser desenhado; se `-1` desenha todos os contornos
* `cor` é a cor do pixel a ser usada para desenhar o contorno, por exemplo, `(255, 0, 0)` para azul. Se for `-1`, o contorno é **preenchido** com a cor.

In [10]:
contornos, arvore = cv2.findContours(mask_coke.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) 
print(f'Numero de Contornos Encontrados: {len(contornos)}')

contornos_img = img.copy()
cv2.drawContours(contornos_img, contornos, -1, [255, 0, 0], 3)

cv2.imshow("contornos_img", contornos_img)
cv2.waitKey()
cv2.destroyAllWindows()

Numero de Contornos Encontrados: 111


## Medidas dos contornos

A partir dos contornos, podemos tirar uma série de medidas como:
- **Área:** número de pixels pertencentes ao contorno, calculada com `cv2.contourArea(contour)`
- **Centro de massa:** linha e coluna do centro de massa do contorno
- **Caixa delimitadora:** menor retângulo que contém o contorno, calculada com `cv2.boundingRect(contour)`

### Maior contorno
Utilizando a função `cv2.contourArea()` podemos calcular a área de cada contorno e assim encontrar o maior contorno.

Na célula abaixo, apresentamos duas formas de encontrar o maior contorno dentre os contornos encontrados na imagem.

In [11]:
import time

## Utilizando laço
start = time.perf_counter()
maior = None
maior_area = 0
for c in contornos:
    area = cv2.contourArea(c)
    if area > maior_area:
        maior_area = area
        maior = c

print(f"Tempo de execução: {time.perf_counter() - start:.5f}s")

## Utilizando max e key
start = time.perf_counter()
maior = max(contornos, key=cv2.contourArea)
print(f"Tempo de execução: {time.perf_counter() - start:.5f}s")

Tempo de execução: 0.00154s
Tempo de execução: 0.00037s


In [15]:
contornos_img = img.copy()
cv2.drawContours(contornos_img, [maior], -1, [255, 0, 0], 3)

cv2.imshow("contornos_img", contornos_img)
cv2.waitKey()
cv2.destroyAllWindows()

### Centro de massa do contorno.

O centro de massa de um contorno é calculado através da função `cv2.moments(contour)`, que retorna um dicionário com as seguintes chaves:
* `m00`: área do contorno
* `m10`: soma das coordenadas x dos pixels do contorno
* `m01`: soma das coordenadas y dos pixels do contorno

Essas chaves são usadas para calcular o centro de massa do contorno, que é dado por:
```
cX = int(M["m10"] / M["m00"])
cY = int(M["m01"] / M["m00"])
```

Além disso, a função `cv2.moments(contour)` também retorna outras chaves que podem ser usadas para calcular outras medidas do contorno. Essas chaves não serão usadas no curso, mas são apresentadas abaixo para referência:

* `m20`: soma das coordenadas x^2 dos pixels do contorno
* `m11`: soma das coordenadas x*y dos pixels do contorno
* `m02`: soma das coordenadas y^2 dos pixels do contorno
* `m30`: soma das coordenadas x^3 dos pixels do contorno
* `m21`: soma das coordenadas x^2*y dos pixels do contorno
* `m12`: soma das coordenadas x*y^2 dos pixels do contorno
* `m03`: soma das coordenadas y^3 dos pixels do contorno


In [17]:
def crosshair(img, point, size, color):
    """ Desenha um crosshair centrado no point.
        point deve ser uma tupla (x,y)
        color é uma tupla R,G,B uint8
    """
    x,y = point
    cv2.line(img,(x - size,y),(x + size,y),color,5)
    cv2.line(img,(x,y - size),(x, y + size),color,5)

    return img

""" Retorna uma tupla (cx, cy) que desenha o centro do contorno"""
M = cv2.moments(maior)
# Usando a expressão do centróide definida em: https://en.wikipedia.org/wiki/Image_moment
cX = int(M["m10"] / M["m00"])
cY = int(M["m01"] / M["m00"])

contornos_img = crosshair(contornos_img, (cX,cY), 10, (255,0,0))

cv2.imshow("contornos_img", contornos_img)
cv2.waitKey()
cv2.destroyAllWindows()

#### Caixa delimitadora

Em visão computacional, uma caixa delimitadora é um retângulo que delimita as coordenadas de um objeto. A caixa delimitadora é definida pelas coordenadas de seu canto **superior esquerdo** e sua **largura e altura**.

Vamos calcular a caixa delimitadora da maior latinha de refrigerante na imagem.

In [18]:
# get bounding rect
x, y, w, h = cv2.boundingRect(maior)

cv2.rectangle(contornos_img, (x, y), (x+w, y+h), (0, 255, 0), 2)

cv2.imshow("contornos_img", contornos_img)
cv2.waitKey()
cv2.destroyAllWindows()

# Prática

**Exercício 1**: ainda trabalhando com as imagens das latinhas, use seus conhecimentos de segmentação de imagens para obter uma máscara que tenha somente o contorno externo das 4 latinhas.

**Dica:** Pode utilizar a função `sorted´ para ordenar os contornos por área.

**Exercício 2**: usando código, faça uma função que filtra a máscara anterior e deixa somente as latinhas da Coca-cola Life (a que tem a parte verde no topo).

**Dica**: use as funções de contornos para analisar cada latinha individualmente. Depois disso, veja se a latinha contém uma quantidade de pixels verdes "grande".

**Exercício 3:** Abra a imagem `pingpong.jpg` e desenhe o centro de massa e a caixa delimitadora da bolinha laranja. Imprima também a sua área na tela.

<div id="hough"></div>